# Sandbox para seguir en vivo las clases

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

## 2020-04-14

In [40]:
def sir_sim(infected_zero=0.001, g_zero=3, total_time=50, alpha=20):
    infected_s = [infected_zero]
    removed_s = [infected_zero]
    infected_h = infected.copy()
    removed_h = removed_s.copy()
    
    transmition_rate_s = g_zero * (1 + alpha * infected_s[0]) ** (-1)
    transmition_rate_h = g_zero * (1 + alpha * removed_h[0]) ** (-1)
    
    for week in range(1, total_time):
        new_infections_s = transmition_rate_s * infected_s[week-1] * (1 - removed_s[week-1])
        new_infections_h = transmition_rate_h * infected_h[week-1] * (1 - removed_h[week-1])
        
        infected_s.append(new_infections_s)
        infected_h.append(new_infections_h)
        
        removed_s.append(removed_s[week-1] + new_infections_s)
        removed_h.append(removed_h[week-1] + new_infections_h)
        
        transmition_rate_s = g_zero / (1 + alpha * infected_s[week])
        transmition_rate_h = g_zero / (1 + alpha * removed_h[week])

    time_axis = np.arange(0, total_time)
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=time_axis, y=infected_s, mode='lines', name='infected s.s.')
    )
    fig.add_trace(
        go.Scatter(x=time_axis, y=infected_h, mode='lines', name='removed h.a.')
    )
    fig.show()

sir_sim()

## 2020-04-09

In [ ]:
# Diferencias centradas vs forward
def f(x):
    return abs(x-1)

def f2(x):
    return x ** 2

def f3(x):
    return x ** 3

h = np.logspace(-20, 0, 10000)  # numpy


x0 = 1
Dh = (f(x0+h) - f(x0)) / h
Dh2 = f(x0+h) - f(x0-h) / 2*h
plt.loglog(x0, Dh)